In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
conn = sqlite3.connect("/kaggle/input/188-million-us-wildfires/FPA_FOD_20170508.sqlite")
data = pd.read_sql("select * from fires",conn)
conn.close()


In [ ]:
print(list(data.columns))
# COMPLEX_NAME maybe worth checking
clean_data_v1 = data.drop(['OBJECTID', 'FOD_ID', 'FPA_ID','SOURCE_SYSTEM','SOURCE_SYSTEM_TYPE',
                          'NWCG_REPORTING_UNIT_ID','NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT','SOURCE_REPORTING_UNIT_NAME',
                          'LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID','FIRE_CODE','FIRE_NAME','ICS_209_INCIDENT_NUMBER','ICS_209_NAME',
                          'MTBS_ID','MTBS_FIRE_NAME','COMPLEX_NAME','FIRE_SIZE_CLASS','Shape','FIPS_CODE','FIPS_NAME'],axis=1)

In [ ]:
clean_data_v1.describe()


In [ ]:
clean_data_v1.info()

In [ ]:
## CONT_DATE - DISCOVERY_DATE = FIRE_PERIOD

df = pd.DataFrame(clean_data_v1['CONT_DATE'])
df['DISC_DATE'] = clean_data_v1['DISCOVERY_DATE'] # create df with cont day and discorvery day
df['NANS'] = df['CONT_DATE'].isnull()   # create column with true false according to nans value
df['NANS'] = df['NANS'].where(df['NANS']==False, df['DISC_DATE']) # replace True values by the discovery day
df['CONT_DATE'] = df['CONT_DATE'].fillna(0)  # replace nans values by the 0 in cont day
df['CONT_DATE'] = df['CONT_DATE'].where(df['CONT_DATE']!=0, df['NANS']) # replace 0 values by the discovery day in cont day column
final_df_time = df.drop(['NANS'], axis=1)
final_df_time

# print((clean_data_v1['CONT_DATE']).size-((clean_data_v1['CONT_DATE']).dropna()).size)
# print(len(df))

In [ ]:
# BOX SHIR & LOLA 23/11 + 24/11

import datetime as dt
import math

# CYCLIC COLUMNS FOR DAY OF YEAR
day_disc = clean_data_v1['DISCOVERY_DOY']
day_norm = 2 * math.pi * day_disc / day_disc.max()
clean_data_v1["cyclic_day_of_year"] = np.cos(day_norm)
clean_data_v1.drop(['DISCOVERY_DOY'], axis=1)

# CYCLIC COLUMNS FOR DAY OF WEEK 
discovery_date = pd.to_datetime(clean_data_v1['DISCOVERY_DATE'], unit="D", origin="julian")
day_of_week = discovery_date.dt.dayofweek
week_day_norm = 2 * math.pi * day_of_week
clean_data_v1["cyclic_day_of_week"] = np.cos(day_of_week / 6)
clean_data_v1.drop(['DISCOVERY_DATE'], axis=1)

# CYCLIC COLUMNS FOR MONTH 
month_of_year = pd.DatetimeIndex(discovery_date).month
month_of_year_norm = 2 * math.pi * (month_of_year-1)
clean_data_v1["cyclic_month_of_year"] = np.cos(month_of_year_norm / 11)

# ADD FEATURE WEEK END - SEASON

# HOUR Feature
detection_hours_array = pd.DataFrame(clean_data_v1['CONT_TIME']).fillna('1400')
detection_hours_array['CONT_TIME']= pd.to_numeric(detection_hours_array['CONT_TIME'])
hours = pd.DataFrame(detection_hours_array['CONT_TIME']/100)
hours = hours.fillna(14.00)
hours = hours.replace([np.inf, -np.inf], 14.00)
hours['CONT_TIME'] = hours['CONT_TIME'].astype(int)

# 'DISCOVERY_TIME'
disc_hours = pd.DataFrame(clean_data_v1['DISCOVERY_TIME'])
disc_hours['DISCOVERY_TIME']= pd.to_numeric(disc_hours['DISCOVERY_TIME'])

# CONT_DATE - DISCOVERY_DATE = FIRE_PERIOD in term of days
cont_date = pd.to_datetime(final_df_time['CONT_DATE'], unit="D", origin="julian")
# clean_data_v1['fire_period'] = cont_date-discovery_date
# clean_data_v1['fire_period'] = clean_data_v1['fire_period'].dt.days

# CONT_DATE - DISCOVERY_DATE = FIRE_PERIOD in term of hours
time_df = pd.DataFrame(discovery_date)
time_df['Disc_time'] = clean_data_v1['DISCOVERY_TIME'].fillna('1400')
time_df['cont_date'] = cont_date
detection_hours_array = pd.DataFrame(clean_data_v1['CONT_TIME']).fillna('1400')
detection_hours_array['CONT_TIME'] = detection_hours_array['CONT_TIME'].replace('', '1400')
time_df['cont_time'] = detection_hours_array['CONT_TIME']
time_df


In [ ]:
# FIRE PERIOD in term of hours (2)
# time_df['Disc_time'] = pd.to_datetime(time_df['Disc_time'], format='%H%M').dt.time
# time_df['cont_time'] = pd.to_datetime(time_df['cont_time'], format='%H%M').dt.time

In [ ]:
# FIRE PERIOD in term of hours (3)
date = time_df['DISCOVERY_DATE'].dt.strftime('%Y-%m-%d')
time_data_disc = pd.DataFrame(pd.to_datetime(date + ' ' + time_df['Disc_time']))
date_2 = time_df['cont_date'].dt.strftime('%Y-%m-%d')
time_data_cont = pd.DataFrame(pd.to_datetime(date_2 + ' ' + time_df['cont_time']))
fire_period = (time_data_cont - time_data_disc).astype('timedelta64[h]')
fire_period[fire_period < 0] = 0

In [ ]:
# ODED's CODE
# Geographical Feature
# x = np.cos(data["LATITUDE"]) * np.cos(data["LONGITUDE"])
# y = np.cos(data["LATITUDE"]) * np.sin(data["LONGITUDE"])
# z = np.sin(data["LATITUDE"])

# # State feature
# states_dummies = pd.get_dummies(data['STATE'].unique())

# Holidays feature
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
df_holidays = data[['STAT_CAUSE_DESCR', 'DISCOVERY_DATE']].reset_index()
# discovery_date = pd.to_datetime(clean_data_v1['DISCOVERY_DATE'], unit="D", origin="julian")
# discovery_date = 
# df_holidays['DISCOVERY_DATE'] = pd.DataFrame(discovery_date)

cal = calendar()
# print("number of holidays:", len(cal.rules))

holidays_dates = cal.holidays(start=discovery_date.min(), end=discovery_date.max())
df_holidays['is_holiday'] = discovery_date.isin(holidays_dates)
df_holidays

# holidays_which = cal.holidays(start=discovery_date.min(), end=discovery_date.max(), 
#                               return_name=True).reset_index().rename(columns={"index": "date", 0: "holiday"})
# holidays_mapping = dict(zip(holidays_which['date'],holidays_which['holiday']))
# df_holidays['which_holiday'] = df_holidays['DISCOVERY_DATE'].map(holidays_mapping)

# Holidays Graph (1)
# g = sns.catplot(x='STAT_CAUSE_DESCR', y="is_holiday",data=df_holidays,kind="bar")
# g.set_xticklabels(rotation=90)
# g.set(xlabel="", ylabel = "number of wildfires (%")
# g

In [ ]:
# Holidays Graph (2)
# g =  sns.catplot(x="which_holiday", col="STAT_CAUSE_DESCR",data=df_holidays, kind="count", col_wrap=2,height=6, ci=None )
# g.set_xticklabels(rotation=90)
# g

In [ ]:
# Size Class Feature
size_class_dummies = pd.get_dummies(data['FIRE_SIZE_CLASS'])

In [ ]:
#  TAL BOX 3
# Create 13 features for each reason
# a = data['STATE']
# b = data['STAT_CAUSE_DESCR']
# sum_cause_state_dict = {} # count for each state and each reason 

# for i in range(a.shape[0]):
#     if tuple((a[i],b[i])) not in sum_cause_state_dict:
#         sum_cause_state_dict[tuple((a[i],b[i]))] = 1
#     else:
#         sum_cause_state_dict[tuple((a[i],b[i]))] += 1

In [ ]:
# TAL BOX 4
# state_codes = set(data['STATE'])
# causes = list(set(data['STAT_CAUSE_DESCR']))
# for code in state_codes:
#     for cause in causes:
#         if tuple((code,cause)) not in sum_cause_state_dict:
#             sum_cause_state_dict[tuple((code,cause))] = 0

# # Create the 13 features 
# for cause in causes:
#     data['state_' +  cause + '_rate'] = np.zeros(data.shape[0],dtype='float64')

In [ ]:
# TAL BOX 5
# Relation Matrix between state and reason 
# ratio_mat = np.ndarray((data.shape[0],len(causes)),dtype='float64')

# state_ratio_dict = {}
# for state in state_codes:
#     ratios = np.zeros(len(causes), dtype='float64')
#     for i in range(len(causes)):
#         ratios[i] = sum_cause_state_dict[tuple((state,causes[i]))]
#     ratios = ratios / data[data['STATE'] == state].shape[0]
#     state_ratio_dict[state] = ratios

In [ ]:
# TAL BOX 6
# Filling Relation matrix
# for i in range(data.shape[0]):
#     ratio_mat[i] = state_ratio_dict[data['STATE'][i]]
    
# for i in range(len(causes)):
#     data['state_' +  causes[i] + '_rate'] = ratio_mat[:, i]

In [ ]:
# TAL BOX 7 
# Same processus with MATRIX ratios
# d = data['OWNER_DESCR']
# sum_owner_dict = {}
# for e in d:
#     if e not in sum_owner_dict:
#         sum_owner_dict[e] = 1
#     else:
#         sum_owner_dict[e] += 1
# fires_per_owner = np.zeros(d.shape[0])
# data['fires_owner'] = [sum_owner_dict[val] for val in d]

In [ ]:
# TAL BOX 8 
# a = data['OWNER_DESCR']
# b = data['STAT_CAUSE_DESCR']
# sum_cause_owner_dict = {}
# for i in range(a.shape[0]):
#     if tuple((a[i],b[i])) not in sum_cause_owner_dict:
#         sum_cause_owner_dict[tuple((a[i],b[i]))] = 1
#     else:
#         sum_cause_owner_dict[tuple((a[i],b[i]))] += 1

In [ ]:
# TAL BOX 9
# owner_codes = set(data['OWNER_DESCR'])
# causes = list(set(data['STAT_CAUSE_DESCR']))
# for code in owner_codes:
#     for cause in causes:
#         if tuple((code,cause)) not in sum_cause_owner_dict:
#             sum_cause_owner_dict[tuple((code,cause))] = 0

In [ ]:
# TAL BOX 10
# ratio_mat = np.ndarray((data.shape[0],len(causes)),dtype='float64')
# owner_ratio_dict = {}
# for owner in owner_codes:
#     ratios = np.zeros(len(causes), dtype='float64')
#     for i in range(len(causes)):
#         ratios[i] = sum_cause_owner_dict[tuple((owner,causes[i]))]
#     ratios = ratios / data[data['OWNER_DESCR'] == owner].shape[0]
#     owner_ratio_dict[owner] = ratios

# for i in range(data.shape[0]):
#     ratio_mat[i] = owner_ratio_dict[data['OWNER_DESCR'][i]]
# for i in range(len(causes)):
#     data['owner_' +  causes[i] + '_rate'] = ratio_mat[:, i]

In [ ]:
# TAL BOX 11
# Same processus for both features : Owner Description & States VS Reasons
# Overfitted Feature
grouped = data[['OWNER_DESCR','STATE','STAT_CAUSE_DESCR']].groupby(by=['OWNER_DESCR','STATE','STAT_CAUSE_DESCR'])

owner_state_cause_dict = {}
owners = set(data['OWNER_DESCR'])
states = set(data['STATE'])
causes = set(data['STAT_CAUSE_DESCR'])
             
for owner in owners:
    for state in states:
        for cause in causes:
            owner_state_cause_dict[tuple((owner,state,cause))] = 0
for x,y in grouped:
    owner_state_cause_dict[tuple((x[0],x[1],x[2]))] = y.shape[0]

In [ ]:
# Tal box 12
grouped = data[['OWNER_DESCR','STATE']].groupby(by=['OWNER_DESCR','STATE'])
owner_state_dict = {}
owners = set(data['OWNER_DESCR'])
states = set(data['STATE'])
             
# For not divide by 0
for owner in owners:
    for state in states:
        owner_state_dict[tuple((owner, state))] = 1
len(owner_state_dict)

for x,y in grouped:
    owner_state_dict[tuple((x[0], x[1]))] = y.shape[0]

In [ ]:
# LAST BOX TAL
ratio_mat = np.ndarray((data.shape[0],len(causes)),dtype='float64')
owner_state_ratio_dict = {}
causes = list(causes)
for owner in owners:
    for state in states:
        ratios = np.zeros(len(causes), dtype='float64')
        sumi = 0
        for i in range(len(causes)):
            ratios[i] = owner_state_cause_dict[tuple((owner,state,causes[i]))]
            sumi += owner_state_cause_dict[tuple((owner,state,causes[i]))]
        ratios = ratios / owner_state_dict[tuple((owner,state))]
        owner_state_ratio_dict[tuple((owner,state))] = ratios
        
for i in range(data.shape[0]):
    owner = data['OWNER_DESCR'][i]
    state = data['STATE'][i]
    ratio_mat[i] = owner_state_ratio_dict[tuple((owner,state))]
    
a = pd.DataFrame(data[['OWNER_DESCR','STATE','STAT_CAUSE_DESCR']])
for i in range(len(causes)):
    data['owner_state_' +  causes[i] + '_rate'] = ratio_mat[:, i]
    a['owner_state_' +  causes[i] + '_rate'] = ratio_mat[:, i]

In [ ]:
# Geographic feature
import geopandas as gpd
import shapely
from shapely.geometry import Point # Point class
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface
import gc

a = data[['LATITUDE','LONGITUDE']]
fires = gpd.GeoDataFrame(a.drop(['LATITUDE', 'LONGITUDE'], axis=1),
                       crs={'init': 'epsg:4326'},
                       geometry=[shapely.geometry.Point(xy)
                                 for xy in zip(a.LONGITUDE, a.LATITUDE)])
cities = gpd.read_file('/kaggle/input/cities/tl_2017_us_uac10.shp')

fires_in_cities = gpd.sjoin(fires, cities, how="left", op='within')
a = fires_in_cities['FUNCSTAT10'].replace('S',1)
a = a.replace(np.nan,0)
del cities
gc.collect()

In [ ]:
# TAL BOX 2
d = data['STATE'] 
sum_state_dict = {} # number of fires in each state
for e in d:
    if e not in sum_state_dict:
        sum_state_dict[e] = 1
    else:
        sum_state_dict[e] += 1
fires_per_state = np.zeros(d.shape[0])
data['fires_in_state'] = [sum_state_dict[val] for val in d] # new feature

In [ ]:
# FINAL DATA 
label = clean_data_v1['STAT_CAUSE_DESCR']

final_df = clean_data_v1["cyclic_month_of_year"]
final_df = pd.concat([final_df, size_class_dummies], axis=1)
final_df = pd.concat([final_df, data.iloc[:, 39:52]], axis=1)
final_df['cyclic_day_of_week'] = clean_data_v1['cyclic_day_of_week']
final_df['cyclic_day_of_year'] = clean_data_v1['cyclic_day_of_year'] 
final_df['hour_in_day'] = hours['CONT_TIME']
final_df['fire_period'] = fire_period
final_df['is_urban'] = a
final_df['is_holidays'] = df_holidays['is_holiday'].astype(int)
final_df['num_fires_in_state'] = data['fires_in_state']
final_df

**MODELS**

In [ ]:
from sklearn.model_selection import train_test_split

#Import scikit-learn metrics module for accuracy calculation 
from sklearn import metrics 
from sklearn.metrics import zero_one_loss 
from sklearn.model_selection import GridSearchCV 

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegressionCV 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import average_precision_score

np.random.seed(137)

label=data['STAT_CAUSE_DESCR']
X_train, X_test, y_train, y_test = train_test_split(final_df, label, test_size=0.25) # 75% training and 25% test

#Create a Gaussian Classifier 
clf = xgb.XGBClassifier(max_depth=22,n_estimators=150)
# clf = RandomForestClassifier(max_depth=20, n_estimators = 150)
clf.fit(X_train,y_train)

# X_train.info()
#Train the model using the training sets 
y_pred=clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# # average_precision = dict()
# n_classes = 7
# for i in range(n_classes):
#     average_precision[i] = average_precision_score(y_test[:, i], y_pred[:, i])

# A "micro-average": quantifying score on all classes jointly
# average_precision["micro"] = average_precision_score(y_test, y_pred,
#                                                      average="micro")
# print('Average precision score, micro-averaged over all classes: {0:0.2f}'
#       .format(average_precision["micro"]))
# y_pred


In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred, average='macro')

In [ ]:
# import catboost as cb
# import numpy as np
# from sklearn.datasets import load_breast_cancer
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split

# import optuna


# def objective(trial):
#     label=data['STAT_CAUSE_CODE']
#     train_x, valid_x, train_y, valid_y = train_test_split(final_df, label, test_size=0.3)

#     param = {
#         "objective": trial.suggest_categorical("objective", ["MultiClass"]),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
#         "depth": trial.suggest_int("depth", 1, 12),
#         "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
#         "bootstrap_type": trial.suggest_categorical(
#             "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
#         )
#     }

#     if param["bootstrap_type"] == "Bayesian":
#         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
#     elif param["bootstrap_type"] == "Bernoulli":
#         param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

#     gbm = cb.CatBoostClassifier(**param)
#     print('before fit')
#     gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
#     print('after fit')

#     preds = gbm.predict(valid_x)
#     pred_labels = np.rint(preds)
#     accuracy = accuracy_score(valid_y, pred_labels)
#     print(accuracy)
#     print("Precision:", precision_score(valid_y, pred_labels, average='macro'))
#     return accuracy



# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10, show_progress_bar=True)

# print("Number of finished trials: {}".format(len(study.trials)))

# print("Best trial:")
# trial = study.best_trial

# print("  Value: {}".format(trial.value))

# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [ ]:
# # Gradient Boosting Model
# from sklearn.model_selection import train_test_split
# from sklearn import metrics 
# from sklearn.ensemble import GradientBoostingClassifier 
# from sklearn.model_selection import cross_val_score
# import optuna 
# from optuna import trial
# from sklearn.metrics import precision_score
# from sklearn.pipeline import Pipeline

# label=data['STAT_CAUSE_DESCR']
# X_train, X_test, y_train, y_test = train_test_split(final_df, label, test_size=0.25)
# # model = Pipeline([('gbm',GradientBoostingClassifier())])

# def objective(trial):

#     n_estimators =  trial.suggest_int('n_estimators', 2, 20)
# #     max_features = trial.suggest_int('max_features', 1,20)
# #     subsample = trial.suggest_uniform('subsample', 0.1, 0.5)
    
#     params = {
#             'n_estimators': n_estimators
# #             'max_features': max_features,
# #             'subsample': subsample
#             }
#     model = GradientBoostingClassifier()
#     model.set_params(**params)
#     return - np.mean(cross_val_score(model, X_train, y_train, cv=8))

# def Optimized_GradientBoostingClassifier():
#     # optimize hyperparameters
#     study = optuna.create_study(direction='maximize')
#     study = study.optimize(objective, n_trials=2, timeout=600, show_progress_bar=True)

#     model.set_params(**study.best_params)
#     model.fit(X_train, y_train)
#     y_pred=model.predict(X_test)
#     print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#     print("Precision:", precision_score(y_test, y_pred, average='macro'))

# # def XGB():
# #     clf = xgboost.XGBClassifier().fit(X_train, y_train)
# Optimized_GradientBoostingClassifier()

In [ ]:
# import catboost as cb
# import numpy as np
# from sklearn.datasets import load_breast_cancer
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split

# import optuna


# def objective(trial):
#     label=data['STAT_CAUSE_CODE']
#     train_x, valid_x, train_y, valid_y = train_test_split(final_df, label, test_size=0.3)

#     param = {
#         "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
#         "depth": trial.suggest_int("depth", 1, 12),
#         "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
#         "bootstrap_type": trial.suggest_categorical(
#             "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
#         ),
#         "used_ram_limit": "3gb",
#         "loss_function" : 'MultiClass'
#     }

#     if param["bootstrap_type"] == "Bayesian":
#         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
#     elif param["bootstrap_type"] == "Bernoulli":
#         param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

#     gbm = cb.CatBoostClassifier(**param)

#     gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

#     preds = gbm.predict(valid_x)
#     pred_labels = np.rint(preds)
#     accuracy = accuracy_score(valid_y, pred_labels)
#     return accuracy


# if __name__ == "__main__":
#     study = optuna.create_study(direction="maximize")
#     study.optimize(objective, n_trials=10, timeout=600, show_progress_bar=True)

#     print("Number of finished trials: {}".format(len(study.trials)))

#     print("Best trial:")
#     trial = study.best_trial

#     print("  Value: {}".format(trial.value))

#     print("  Params: ")
#     for key, value in trial.params.items():
#         print("    {}: {}".format(key, value))

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# from datetime import datetime
# import statistics
# clean_data_heat = clean_data_v1[['FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME',
#        'STAT_CAUSE_CODE', 'CONT_DATE', 'CONT_DOY',
#        'CONT_TIME', 'FIRE_SIZE', 'LATITUDE', 'LONGITUDE']]
# heat_names =['FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME',
#        'STAT_CAUSE_CODE', 'CONT_DATE', 'CONT_DOY',
#        'CONT_TIME', 'FIRE_SIZE', 'LATITUDE', 'LONGITUDE']

# ###
# detection_hours_array = np.array(np.array((clean_data_heat['CONT_TIME']).dropna(),dtype ='int32')/100,dtype ='int32')
# detection_hours_mode = statistics.mode(detection_hours_array)
# ###
# print(set(((clean_data_heat['CONT_TIME']).dropna())))
# ###
# contained_hours_array = np.array(np.array((clean_data_heat['DISCOVERY_TIME']).dropna(),dtype ='int32')/100,dtype ='int32')
# contained_hours_mode = statistics.mode(contained_hours_array)
# ###
# cur = pd.DataFrame(detection_hours_array,columns=['hour'])
# a = sns.countplot('hour',data=cur)
# a.set_xticklabels(set(cur['hour']),rotation=45)
# # 

# # for cont_date 
# #     df_test['Difference'] = df_test['First_Date'].sub(df_test['Second Date'], axis=0)

# #     df_test['CONT_DAY'] = df_test['DISCOVERY_DATE'].sub(df_test['CONT_DATE'], axis=0)



# clean_data_v1 = clean_data_v1.dropna()
# f,ax=plt.subplots(1,1,figsize=(20,10))
# sns.heatmap(np.abs(clean_data_v1.corr()),annot=True,cmap='RdYlGn',ax=ax)

